<a href="https://colab.research.google.com/github/gocgodman/M2M/blob/main/M2M(mp3%20to%20midi).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# 셀 1: 필요한 패키지 설치 (한 번만 실행)
# Transkun v2 전사기 설치 (PyPI 패키지명이 다르면 수정)
!pip install -U transkun

# PyTorch (GPU 런타임일 경우 CUDA 맞는 버전 사용)
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

# 기타 유틸
!pip install -q yt-dlp gdown gradio pretty_midi librosa numpy soundfile pyfluidsynth
!apt-get update -qq
!apt-get install -y -qq ffmpeg fluidsynth p7zip-full || true

# librosa 버전 고정(필요시)
!pip install -q librosa==0.9.2 --upgrade

In [ ]:
# ============================================================================
# 셀 2: Google Drive 마운트 및 경로 설정
# ============================================================================

from google.colab import drive
drive.mount('/content/drive', force_remount=False)

import os, glob, shutil, zipfile, tarfile, subprocess
import gdown

# 기본 경로 설정
WORK_ROOT = "/content/ytd_pipeline_work"
DRIVE_SF2_DIR = "/content/drive/MyDrive/sf2_library"
DRIVE_RESULTS_DIR = "/content/drive/MyDrive/ytd_pipeline_results"
STATE_FILE = os.path.join(DRIVE_RESULTS_DIR, "pipeline_state.json")

os.makedirs(WORK_ROOT, exist_ok=True)
os.makedirs(DRIVE_SF2_DIR, exist_ok=True)
os.makedirs(DRIVE_RESULTS_DIR, exist_ok=True)

print("WORK_ROOT:", WORK_ROOT)
print("DRIVE_SF2_DIR:", DRIVE_SF2_DIR)
print("DRIVE_RESULTS_DIR:", DRIVE_RESULTS_DIR)

# 공유 폴더에서 SF2 다운로드 (선택사항)
SHARED_FOLDER_ID = "1JkTMvPwM_XURqG2114n4Qj0rR83WEucL"
OUT_DIR = "/content/sf2_from_shared"
os.makedirs(OUT_DIR, exist_ok=True)

if SHARED_FOLDER_ID:
    try:
        gdown.download_folder(id=SHARED_FOLDER_ID, output=OUT_DIR, quiet=False)
        print("✓ 공유 폴더 다운로드 완료")
    except Exception as e:
        print("공유 폴더 다운로드 실패:", e)

# 압축 해제 함수
def try_extract_archive(path, dest):
    """압축 파일 자동 해제"""
    path_lower = path.lower()
    os.makedirs(dest, exist_ok=True)
    try:
        if path_lower.endswith(".zip"):
            with zipfile.ZipFile(path, 'r') as zf:
                zf.extractall(dest)
                return True
        if path_lower.endswith((".tar.gz", ".tgz", ".tar")):
            with tarfile.open(path, 'r:*') as tf:
                tf.extractall(dest)
                return True
        if path_lower.endswith(".7z"):
            cmd = ['7z', 'x', '-y', '-o' + dest, path]
            subprocess.run(cmd, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            return True
    except Exception as e:
        print("압축 해제 실패:", e)
    return False

# SF2 파일 압축 해제
print("\n[SF2 압축 파일 해제 중...]")
for root, dirs, files in os.walk(DRIVE_SF2_DIR):
    for fn in files:
        if fn.lower().endswith((".zip", ".tar.gz", ".tgz", ".tar", ".7z")):
            extract_dest = os.path.join(root, fn + "_extracted")
            if try_extract_archive(os.path.join(root, fn), extract_dest):
                print(f"  ✓ {fn}")

for root, dirs, files in os.walk(OUT_DIR):
    for fn in files:
        if fn.lower().endswith((".zip", ".tar.gz", ".tgz", ".tar", ".7z")):
            extract_dest = os.path.join(root, fn + "_extracted")
            if try_extract_archive(os.path.join(root, fn), extract_dest):
                print(f"  ✓ {fn}")

# SF2 파일 수집 및 복사
local_sf2_files = glob.glob(os.path.join(DRIVE_SF2_DIR, "**/*.sf2"), recursive=True)
local_names = {os.path.basename(f) for f in local_sf2_files}
shared_sf2_files = glob.glob(os.path.join(OUT_DIR, "**/*.sf2"), recursive=True)

print(f"\n내 드라이브 sf2 수: {len(local_sf2_files)}")
print(f"공유 드라이브 sf2 수: {len(shared_sf2_files)}")

# 공유 폴더에서 새 SF2 복사
copied = []
for p in shared_sf2_files:
    fname = os.path.basename(p)
    if fname in local_names:
        continue
    dest = os.path.join(DRIVE_SF2_DIR, fname)
    try:
        if not os.path.exists(dest):
            shutil.copy(p, dest)
            copied.append(dest)
            print(f"  복사: {fname}")
    except Exception as e:
        print(f"  복사 실패: {fname}, {e}")

print(f"새로 복사된 sf2 수: {len(copied)}")

# 잘못 배치된 SF2 정리
misplaced = glob.glob(os.path.join(DRIVE_RESULTS_DIR, "**/*.sf2"), recursive=True)
if misplaced:
    print(f"\n결과 폴더에 잘못 들어간 SF2 수: {len(misplaced)}")
    for p in misplaced:
        fname = os.path.basename(p)
        dst = os.path.join(DRIVE_SF2_DIR, fname)
        try:
            if not os.path.exists(dst):
                shutil.move(p, dst)
                print(f"  이동: {fname}")
            else:
                os.remove(p)
                print(f"  중복 제거: {fname}")
        except Exception as e:
            print(f"  이동/삭제 실패: {fname}, {e}")
else:
    print("\n결과 폴더에 잘못된 SF2 없음")

# 최종 SF2 목록
final_sf2_files = glob.glob(os.path.join(DRIVE_SF2_DIR, "**/*.sf2"), recursive=True)
print(f"\n✓ 최종 SF2 라이브러리 파일 수: {len(final_sf2_files)}")
for f in final_sf2_files[:20]:
    print(f"  * {os.path.basename(f)}")

if len(final_sf2_files) > 20:
    print(f"  ... 외 {len(final_sf2_files) - 20}개")

In [ ]:
# 셀: Transkun v2 전사기 통합
# ================================
# Transkun V2 전사기 통합 (정식)
# ================================
import os, sys, time, uuid, json, glob, shutil, traceback, subprocess
from pathlib import Path

import torch

# -------------------------------
# 외부 툴 체크 유틸 (유지)
# -------------------------------
def run_cmd(cmd, check=False, timeout=None):
    proc = subprocess.run(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True,
        timeout=timeout
    )
    if check and proc.returncode != 0:
        raise RuntimeError(
            f"Command failed: {' '.join(cmd)}\n"
            f"STDOUT:\n{proc.stdout}\n"
            f"STDERR:\n{proc.stderr}"
        )
    return proc.returncode, proc.stdout, proc.stderr

def check_tool(name):
    code, _, _ = run_cmd(["which", name])
    return code == 0

if not check_tool("yt-dlp"):
    print("Warning: yt-dlp not found. YouTube 다운로드가 실패할 수 있습니다.")
if not check_tool("fluidsynth"):
    print("Warning: fluidsynth not found. MIDI -> WAV 렌더링이 동작하지 않을 수 있습니다.")

# -------------------------------
# Transkun V2 정식 API
# -------------------------------
# 설치: pip install -U transkun
try:
    from transkun.transcribe import transcribe
except Exception as e:
    raise RuntimeError(
        "Transkun V2 API import 실패\n"
        "pip install -U transkun 후 런타임 재시작 필요"
    ) from e

# -------------------------------
# 유틸
# -------------------------------
def remove_extension(filepath):
    return os.path.splitext(os.path.basename(filepath))[0]

# -------------------------------
# Transkun V2 전사 함수
# -------------------------------
def transcribe_file(
    input_path,
    outfolder=".",
    device=None
):
    """
    Transkun V2 (Semi-CRF, event-based) 정식 전사
    - input_path: wav / mp3 / flac 등
    - outfolder: midi 출력 폴더
    """

    os.makedirs(outfolder, exist_ok=True)

    base = remove_extension(input_path)
    out_mid = os.path.join(outfolder, base + ".mid")

    if device is None:
        device = "cuda" if torch.cuda.is_available() else "cpu"

    try:
        transcribe(
            audio_path=input_path,
            output_midi_path=out_mid,
            device=device
        )
        return out_mid

    except Exception as e:
        raise RuntimeError(
            f"Transkun V2 transcription failed\n"
            f"Input: {input_path}\n"
            f"Error: {e}\n"
            f"{traceback.format_exc()}"
        )

# -------------------------------
# 사용 예시
# -------------------------------
# mid_path = transcribe_file("piano.wav", outfolder="outputs")
# print("MIDI saved to:", mid_path)
# 페달 규칙 기반 검출 (원본 로직 유지)
def compute_spectral_flux(y, sr, hop_length, n_fft=2048):
    S = np.abs(librosa.stft(y, n_fft=n_fft, hop_length=hop_length))
    S_norm = S / (np.sum(S, axis=0, keepdims=True) + 1e-8)
    flux = np.sqrt(np.sum(np.diff(S_norm, axis=1).clip(min=0)**2, axis=0))
    return np.concatenate(([0.0], flux))

def detect_pedal_rule(audio_path,
                      sr=22050,
                      hop_length=512,
                      low_freq_cut=500,
                      energy_smooth=0.5,
                      on_z=1.0,
                      off_z=0.7,
                      min_event_len=0.06,
                      merge_gap=0.06,
                      closing_size=3):
    y, _ = librosa.load(audio_path, sr=sr, mono=True)
    frame_energy = librosa.feature.rms(y=y, frame_length=2048, hop_length=hop_length)[0]
    S = np.abs(librosa.stft(y, n_fft=2048, hop_length=hop_length))
    freqs = librosa.fft_frequencies(sr=sr, n_fft=2048)
    low_idx = np.where(freqs <= low_freq_cut)[0]
    low_energy = S[low_idx, :].sum(axis=0) if len(low_idx) > 0 else np.zeros_like(frame_energy)
    e = frame_energy / (frame_energy.max() + 1e-8)
    le = low_energy / (low_energy.max() + 1e-8) if low_energy.max() > 0 else low_energy
    combined = 0.6 * le + 0.4 * e
    window = int(max(1, energy_smooth * (sr / hop_length)))
    combined_smooth = uniform_filter1d(combined, size=window)
    mu = combined_smooth.mean(); sigma = combined_smooth.std() + 1e-8
    on_thr = mu + on_z * sigma; off_thr = mu + off_z * sigma
    mask = np.zeros_like(combined_smooth, dtype=bool)
    state = False
    for i, v in enumerate(combined_smooth):
        if not state and v >= on_thr:
            state = True; mask[i] = True
        elif state:
            mask[i] = True
            if v < off_thr:
                state = False
    if closing_size > 1:
        mask = binary_closing(mask, structure=np.ones(closing_size))
    times = librosa.frames_to_time(np.arange(len(mask)), sr=sr, hop_length=hop_length)
    events = []; prev=False; start=None
    for t,m in zip(times, mask):
        if m and not prev: start=t
        if (not m) and prev and start is not None:
            events.append((start,t)); start=None
        prev=m
    if prev and start is not None: events.append((start,times[-1]))
    filtered=[]
    for s,e in events:
        if (e-s) >= min_event_len:
            if filtered and s - filtered[-1][1] <= merge_gap:
                filtered[-1] = (filtered[-1][0], e)
            else:
                filtered.append((s,e))
    return filtered

# 경량 PedalCNN (선택적 보조 모델)
class PedalCNN(nn.Module):
    def __init__(self, in_ch=1):
        super().__init__()
        self.conv1 = nn.Conv1d(in_ch, 16, kernel_size=5, padding=2)
        self.bn1 = nn.BatchNorm1d(16)
        self.conv2 = nn.Conv1d(16, 32, kernel_size=5, padding=2)
        self.bn2 = nn.BatchNorm1d(32)
        self.conv3 = nn.Conv1d(32, 64, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm1d(64)
        self.fc = nn.Linear(64, 1)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.max_pool1d(x, 2)
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.max_pool1d(x, 2)
        x = F.relu(self.bn3(self.conv3(x)))
        x = x.mean(dim=2)
        x = self.fc(x)
        return torch.sigmoid(x).squeeze(-1)

def extract_frame_features(y, sr=22050, hop_length=512, n_mels=40):
    S = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=2048, hop_length=hop_length, n_mels=n_mels)
    logS = librosa.power_to_db(S, ref=np.max)
    return logS  # (n_mels, T)

def predict_pedal_with_model(audio_path, model, device='cpu', sr=22050, hop_length=512, threshold=0.5):
    y, _ = librosa.load(audio_path, sr=sr, mono=True)
    feats = extract_frame_features(y, sr=sr, hop_length=hop_length)  # (n_mels, T)
    X = torch.tensor(feats[np.newaxis, :, :], dtype=torch.float32)  # (1, n_mels, T)
    X = X.mean(dim=1, keepdim=True)  # (1,1,T)
    model.to(device); model.eval()
    with torch.no_grad():
        prob = float(model(X.to(device)).cpu().numpy())
    if prob >= threshold:
        events = detect_pedal_rule(audio_path, sr=sr, hop_length=hop_length)
    else:
        events = []
    return events

def detect_pedal_lightweight(audio_path,
                             use_model=False,
                             model_path=None,
                             device='cpu',
                             sr=22050,
                             hop_length=512,
                             rule_params=None,
                             model_threshold=0.5):
    if rule_params is None:
        rule_params = {}
    rule_events = detect_pedal_rule(audio_path, sr=sr, hop_length=hop_length, **rule_params)
    if not use_model or model_path is None:
        return rule_events
    try:
        model = PedalCNN(in_ch=1)
        ckpt = torch.load(model_path, map_location=device)
        if isinstance(ckpt, dict) and 'state_dict' in ckpt:
            model.load_state_dict(ckpt['state_dict'])
        else:
            model.load_state_dict(ckpt)
        model_events = predict_pedal_with_model(audio_path, model, device=device, hop_length=hop_length, threshold=model_threshold)
        combined = rule_events.copy()
        for me in model_events:
            if not any((abs(me[0]-re[0])<0.05 and abs(me[1]-re[1])<0.05) for re in combined):
                combined.append(me)
        combined = sorted(combined, key=lambda x: x[0])
        merged=[]
        for s,e in combined:
            if not merged:
                merged.append([s,e])
            else:
                if s <= merged[-1][1] + 0.05:
                    merged[-1][1] = max(merged[-1][1], e)
                else:
                    merged.append([s,e])
        return [(s,e) for s,e in merged]
    except Exception:
        return rule_events
# MIDI에 페달 CC 삽입
def insert_pedal_cc_into_midi(midi_in_path, midi_out_path, pedal_events, piano_program=0):
    pm = pretty_midi.PrettyMIDI(midi_in_path)
    for inst in pm.instruments:
        inst.program = piano_program
    target_inst = pm.instruments[0] if pm.instruments else pretty_midi.Instrument(program=piano_program)
    if not pm.instruments:
        pm.instruments.append(target_inst)
    for (s,e) in pedal_events:
        on_time = max(0.0, s - 0.02); off_time = e + 0.02
        target_inst.control_changes.append(pretty_midi.ControlChange(number=64, value=127, time=on_time))
        target_inst.control_changes.append(pretty_midi.ControlChange(number=64, value=0, time=off_time))
    for inst in pm.instruments:
        inst.control_changes.sort(key=lambda cc: cc.time)
    pm.write(midi_out_path)

# fluidsynth 렌더링
def render_midi_to_wav(midi_path, wav_out_path, sf2_path, sample_rate=44100, timeout=120):
    """
    반환: (wav_path or None, render_log string)
    """
    if sf2_path is None or not os.path.exists(sf2_path):
        return None, "sf2_not_found"
    try:
        cmd = ['fluidsynth', '-ni', sf2_path, midi_path, '-F', wav_out_path, '-r', str(sample_rate)]
        proc = subprocess.run(cmd, check=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True, timeout=timeout)
        log = proc.stdout + "\n" + proc.stderr
        if proc.returncode != 0:
            return None, log
        if not os.path.exists(wav_out_path):
            return None, "fluidsynth finished but wav not created\n" + log
        return wav_out_path, log
    except subprocess.TimeoutExpired as e:
        return None, f"fluidsynth timeout: {e}"
    except Exception as e:
        return None, f"fluidsynth exception: {e}\n{traceback.format_exc()}"

# SF2 경로 해석
def resolve_sf2_path(sf2_choice, uploaded_sf2, sf2_library_dir=DRIVE_SF2_DIR):
    # 1) 절대경로 우선
    if sf2_choice and sf2_choice != "None" and os.path.exists(sf2_choice):
        return sf2_choice
    # 2) 업로드된 파일 처리 (업로드 객체가 dict일 경우 'name' 키 사용)
    if uploaded_sf2 and isinstance(uploaded_sf2, list) and len(uploaded_sf2) > 0:
        up = uploaded_sf2[0]
        src = up['name'] if isinstance(up, dict) and 'name' in up else up
        if os.path.exists(src):
            os.makedirs(sf2_library_dir, exist_ok=True)
            dst = os.path.join(sf2_library_dir, os.path.basename(src))
            if not os.path.exists(dst):
                shutil.copy(src, dst)
            return dst
    # 3) 라이브러리 내 후보 반환
    if os.path.exists(sf2_library_dir):
        candidates = [os.path.join(sf2_library_dir, f) for f in os.listdir(sf2_library_dir) if f.lower().endswith('.sf2')]
        if candidates:
            return candidates[0]
    return None

# 상태 관리
STATE_FILE = os.path.join(DRIVE_RESULTS_DIR, "pipeline_state.json")
def load_state():
    if os.path.exists(STATE_FILE):
        with open(STATE_FILE, "r", encoding="utf-8") as f:
            return json.load(f)
    return {"processed": [], "failed": [], "items": {}, "last_update": None}

def save_state(state):
    state["last_update"] = time.time()
    with open(STATE_FILE, "w", encoding="utf-8") as f:
        json.dump(state, f, ensure_ascii=False, indent=2)

# yt-dlp 헬퍼
def expand_playlist_to_video_urls(playlist_url):
    cmd = ["yt-dlp", "--flat-playlist", "-J", playlist_url]
    code, out, err = run_cmd(cmd)
    if code != 0:
        raise RuntimeError(f"yt-dlp playlist expand failed: {err}")
    j = json.loads(out)
    entries = j.get("entries", [])
    urls = []
    for e in entries:
        vid = e.get("id")
        if vid:
            urls.append(f"https://www.youtube.com/watch?v={vid}")
    return urls

def download_youtube_audio_single(url, outdir, fmt="mp3", audio_bitrate="192K"):
    os.makedirs(outdir, exist_ok=True)
    out_template = os.path.join(outdir, "%(playlist_index)s-%(title)s.%(ext)s")
    cmd = ["yt-dlp", "-x", "--audio-format", fmt, "--audio-quality", audio_bitrate, "-o", out_template, url]
    code, out, err = run_cmd(cmd)
    files = sorted(glob.glob(os.path.join(outdir, f"*.{fmt}")))
    if not files:
        raise RuntimeError(f"No audio file produced by yt-dlp for {url}. yt-dlp stderr: {err}")
    return files[-1]

# 통합 항목 처리
def process_item_from_url_or_path(item, sf2_choice=None, uploaded_sf2=None, tmp_dir=None, pedal_model_path=None, use_pedal_model=False):
    """
    item: YouTube URL 또는 로컬 오디오 파일 경로
    반환: dict(status, mp3, midi, wav, pedals, transcribe_log, render_log, error, trace)
    """
    if tmp_dir is None:
        tmp_dir = os.path.join(WORK_ROOT, "tmp_" + uuid.uuid4().hex)
    os.makedirs(tmp_dir, exist_ok=True)

    result = {"status": "error", "mp3": None, "midi": None, "wav": None, "pedals": None, "transcribe_log": None, "render_log": None, "error": None, "trace": None}
    try:
        # 1) 오디오 확보
        if isinstance(item, str) and item.startswith("http"):
            mp3 = download_youtube_audio_single(item, tmp_dir, fmt="mp3")
        else:
            mp3 = item
        result["mp3"] = mp3

        # 2) 전사 (Transkun v2)
        mid, trans_log = transcribe_file(mp3, outfolder=tmp_dir)
        result["transcribe_log"] = trans_log
        result["midi"] = mid

        # 3) 페달 검출
        pedals = detect_pedal_lightweight(mp3, use_model=use_pedal_model, model_path=pedal_model_path, device='cuda' if torch.cuda.is_available() else 'cpu')
        result["pedals"] = pedals

        # 4) MIDI에 CC 삽입
        mid_pedal = os.path.join(tmp_dir, os.path.splitext(os.path.basename(mid))[0] + "_pedal.mid")
        insert_pedal_cc_into_midi(mid, mid_pedal, pedals, piano_program=0)
        result["midi"] = mid_pedal

        # 5) Drive에 저장 (mp3, midi)
        saved_mp3 = os.path.join(DRIVE_RESULTS_DIR, f"{remove_extension(os.path.basename(mp3))}_{uuid.uuid4().hex}.mp3")
        shutil.copy(mp3, saved_mp3)
        result["mp3"] = saved_mp3

        saved_midi = os.path.join(DRIVE_RESULTS_DIR, f"{remove_extension(os.path.basename(mid_pedal))}_{uuid.uuid4().hex}.mid")
        shutil.copy(mid_pedal, saved_midi)
        result["midi"] = saved_midi

        # 6) 렌더링 (선택) - SF2 경로 해석
        chosen_sf2 = resolve_sf2_path(sf2_choice, uploaded_sf2)
        saved_wav = None
        render_log = None
        if chosen_sf2:
            wav_out = os.path.join(tmp_dir, f"{remove_extension(os.path.basename(mp3))}_render_{uuid.uuid4().hex}.wav")
            rendered, render_log = render_midi_to_wav(mid_pedal, wav_out, chosen_sf2)
            if rendered:
                saved_wav = os.path.join(DRIVE_RESULTS_DIR, f"{remove_extension(os.path.basename(rendered))}_{uuid.uuid4().hex}.wav")
                shutil.copy(rendered, saved_wav)
        result["wav"] = saved_wav
        result["render_log"] = render_log

        result["status"] = "ok"
        return result
    except Exception as e:
        result["error"] = str(e)
        result["trace"] = traceback.format_exc()
        return result

In [ ]:

# 수정된 통합 UI 셀 (SF2 경로/업로드 처리 안정화 포함)
import gradio as gr
import os, glob, shutil, traceback

# SF2 라이브러리 디렉터리 (환경에 맞게 변경 가능)
SF2_LIB_DIR = "/content/drive/MyDrive/sf2_library"

# 1) 드라이브에 있는 SF2 절대경로 목록 생성 (라벨, 값) 쌍
def list_sf2_choices(sf2_dir=SF2_LIB_DIR):
    if not os.path.exists(sf2_dir):
        return [("None", "None")]
    files = sorted(glob.glob(os.path.join(sf2_dir, "**/*.sf2"), recursive=True))
    if not files:
        return [("None", "None")]
    # (label, value) 형태로 반환: label은 basename, value는 절대경로
    choices = [("None", "None")] + [(os.path.basename(p), p) for p in files]
    return choices

sf2_choices = list_sf2_choices()

# 2) 업로드된 SF2 처리: 업로드 임시파일을 라이브러리로 복사하고 절대경로 반환
def handle_uploaded_sf2(uploaded_sf2, sf2_library_dir=SF2_LIB_DIR):
    if not uploaded_sf2:
        return None
    up = uploaded_sf2[0] if isinstance(uploaded_sf2, list) else uploaded_sf2
    # Gradio Files may provide dict with 'name' key or direct path
    src = up.get('name') if isinstance(up, dict) and 'name' in up else up
    if not src or not os.path.exists(src):
        return None
    os.makedirs(sf2_library_dir, exist_ok=True)
    dst = os.path.join(sf2_library_dir, os.path.basename(src))
    try:
        if not os.path.exists(dst):
            shutil.copy(src, dst)
        return dst
    except Exception:
        return None

# 3) UI에서 선택된 값과 업로드된 파일을 합쳐 최종 SF2 절대경로 결정
def get_sf2_choice_value(choice, uploaded_sf2, direct_input):
    # uploaded_sf2 우선
    uploaded_path = handle_uploaded_sf2(uploaded_sf2)
    if uploaded_path:
        return uploaded_path
    # direct input (사용자가 직접 경로 입력) 우선
    if direct_input:
        return direct_input if os.path.exists(direct_input) else None
    # dropdown value는 (label,value) 쌍의 value가 넘어오므로 그대로 사용
    if choice and choice != "None":
        return choice if os.path.exists(choice) else None
    return None

# 4) playlist wrapper (기존 제너레이터에 업로드 우선 경로 전달)
def playlist_wrapper_with_upload(playlist_text, sf2_choice_val, uploaded_sf2_val, sf2_direct_input):
    try:
        chosen = get_sf2_choice_value(sf2_choice_val, uploaded_sf2_val, sf2_direct_input)
        yield from playlist_pipeline_generator(playlist_text, chosen)
    except Exception as e:
        yield f"재생목록 처리 중 예외 발생: {e}", None

# 5) 업로드 처리 콜백: chosen_sf2(절대경로)를 process_files에 전달
def on_run_upload(gr_files, sf2_choice_val, uploaded_sf2_val,
                  sustain_tolerance, pedal_energy_smooth, pedal_low_freq_cut,
                  pedal_low_energy_weight, pedal_flux_weight,
                  pedal_on_threshold, pedal_off_threshold,
                  pedal_min_event_len, pedal_merge_gap,
                  sf2_direct_input):
    try:
        chosen_sf2 = get_sf2_choice_value(sf2_choice_val, uploaded_sf2_val, sf2_direct_input)
        chosen_name = os.path.basename(chosen_sf2) if chosen_sf2 else "없음"
        status = f"업로드 처리 시작. SF2: {chosen_name} (경로: {chosen_sf2 or 'None'})"
        # process_files에 절대경로 chosen_sf2만 전달
        final_midi, wav_path = process_files(
            gr_files,
            sf2_choice=chosen_sf2,
            uploaded_sf2=None,   # 이미 처리했으므로 None
            sustain_tolerance=sustain_tolerance,
            pedal_energy_smooth=pedal_energy_smooth,
            pedal_low_freq_cut=pedal_low_freq_cut,
            pedal_low_energy_weight=pedal_low_energy_weight,
            pedal_flux_weight=pedal_flux_weight,
            pedal_on_threshold=pedal_on_threshold,
            pedal_off_threshold=pedal_off_threshold,
            pedal_min_event_len=pedal_min_event_len,
            pedal_merge_gap=pedal_merge_gap
        )
        # 결과가 None일 수 있으니 안전하게 반환
        return status, final_midi if final_midi else None, wav_path if wav_path else None
    except Exception as e:
        tb = traceback.format_exc()
        return f"업로드 처리 중 오류: {e}\n{tb}", None, None

# 6) Gradio Blocks UI (수정된 드롭다운: (label,value) 쌍 사용)
with gr.Blocks() as combined_demo:
    gr.Markdown("## 통합: 파일 업로드 + 재생목록 배치 처리 (공통 SF2 선택)")
    with gr.Row():
        with gr.Column(scale=1):
            sf2_dropdown = gr.Dropdown(choices=sf2_choices, value=sf2_choices[0][1], label="Drive에서 선택할 SF2")
            sf2_upload = gr.Files(file_types=[".sf2"], label="또는 SF2 업로드 (선택)")
            sf2_direct = gr.Textbox(label="(선택) SF2 절대경로 직접 입력", placeholder="/content/drive/MyDrive/sf2_library/Your.sf2")
            gr.Markdown("**설명:** 업로드된 SF2가 있으면 업로드된 파일을 우선 사용합니다. 드롭다운은 Drive의 SF2 절대경로를 value로 가집니다.")
        with gr.Column(scale=2):
            status_box = gr.Textbox(label="전역 상태", interactive=False)
            status_box.value = f"결과 폴더: {DRIVE_RESULTS_DIR if 'DRIVE_RESULTS_DIR' in globals() else '설정 필요'}"

    with gr.Tabs():
        with gr.TabItem("파일 업로드 처리"):
            upload_files = gr.Files(file_types=[".wav", ".mp3"], label="오디오 파일 업로드")
            sustain = gr.Slider(0.05, 0.6, value=0.2, step=0.01, label="Sustain tolerance (note merge, sec)")
            pedal_smooth = gr.Slider(0.1, 1.0, value=0.5, step=0.05, label="Pedal energy smoothing (sec)")
            pedal_lowcut = gr.Slider(200, 2000, value=500, step=50, label="Pedal low-frequency cutoff (Hz)")
            pedal_low_w = gr.Slider(0.0, 1.0, value=0.6, step=0.05, label="Low-energy weight (pedal)")
            pedal_flux_w = gr.Slider(0.0, 1.0, value=0.4, step=0.05, label="Flux weight (pedal)")
            pedal_on = gr.Slider(0.2, 2.0, value=1.0, step=0.1, label="Pedal ON threshold (sigma multiplier)")
            pedal_off = gr.Slider(0.0, 1.5, value=0.7, step=0.05, label="Pedal OFF threshold (sigma multiplier)")
            pedal_min = gr.Slider(0.02, 0.5, value=0.08, step=0.01, label="Min pedal event length (sec)")
            pedal_merge = gr.Slider(0.01, 0.3, value=0.08, step=0.01, label="Merge gap for pedal events (sec)")
            run_upload = gr.Button("업로드 파일 처리 시작")
            upload_result_file = gr.File(label="Download MIDI or ZIP")
            upload_preview = gr.Audio(label="Preview (WAV) - rendered with chosen SF2", type="filepath")

            run_upload.click(
                fn=on_run_upload,
                inputs=[upload_files, sf2_dropdown, sf2_upload,
                        sustain, pedal_smooth, pedal_lowcut, pedal_low_w, pedal_flux_w,
                        pedal_on, pedal_off, pedal_min, pedal_merge, sf2_direct],
                outputs=[status_box, upload_result_file, upload_preview]
            )

        with gr.TabItem("재생목록 배치 처리"):
            playlist_input = gr.Textbox(lines=6, label="재생목록 URL 또는 영상 URL들 (줄바꿈으로 여러개 입력)")
            playlist_sf2_input = gr.Textbox(label="(선택) SF2 경로를 직접 입력하거나 위 드롭다운/업로드 사용")
            run_playlist_btn = gr.Button("재생목록 처리 시작")
            playlist_log = gr.Textbox(label="진행 로그")
            playlist_last = gr.JSON(label="마지막 항목 결과")

            run_playlist_btn.click(
                fn=playlist_wrapper_with_upload,
                inputs=[playlist_input, sf2_dropdown, sf2_upload, playlist_sf2_input],
                outputs=[playlist_log, playlist_last]
            )

    gr.Markdown("**사용법:** 상단에서 SF2를 드롭다운으로 선택하거나 SF2 파일을 업로드하세요. 업로드 탭은 로컬 파일을 처리하고, 재생목록 탭은 유튜브 재생목록/URL을 배치 처리합니다. 처리 결과는 Drive의 ytd_pipeline_results에 저장됩니다.")

combined_demo.launch(share=True, debug=True)

# Task
The `transkun` package import failed. To debug this, I'll examine the installed `transkun` package to identify the correct import path for the `Transkun` class, `SAMPLE_RATE`, and `load_audio`.

## check_transkun_imports

### Subtask:
Examine the output of the modified cell to identify the correct import path for the 'Transkun' class, 'SAMPLE_RATE', and 'load_audio' from the 'transkun' package.


## Summary:

### Data Analysis Key Findings
- The `transkun` package import failed, indicating an issue with accessing its components.
- The immediate objective is to identify the correct import paths for the `Transkun` class, `SAMPLE_RATE`, and `load_audio` from the `transkun` package to resolve the import error.

### Insights or Next Steps
- The next step is to examine the output of a modified cell specifically designed to reveal the correct import structure within the `transkun` package.
